In [1]:
import pandas as pd

Combine three datasets: 'list_bbox.txt', 'list_category_img.txt', 'list_landmarks.txt' into 1 df

This dataframe 'df' contains the image location/name, box coordinates, category labels and landmark locations (up to 8 locations as coordinates if available)

Category labels:
1) Upper body
2) Lower body
3) Full body

There is an additional column name in the 'landmarks' dataset (variation_type). The values themselves look like they don't actually have this column


In [2]:
dic = {}
for i in ['list_bbox.txt', 'list_category_img.txt', 'list_landmarks.txt']:
    if i == 'list_landmarks.txt':
        cols = pd.read_csv('list_landmarks.txt', sep=r'\s+', skiprows=1, chunksize=1).__next__().columns.to_list()
        cols.remove('variation_type')
        df = pd.read_csv('list_landmarks.txt', sep=r'\s+', skiprows=1)
        df.drop(df.columns[-1], axis=1, inplace=True)
        df.columns = cols
        dic[i] = df
    else:
        dic[i] = pd.read_csv(i, sep=r'\s+', skiprows=1)

list_bbox.txt
list_category_img.txt
list_landmarks.txt


In [3]:
dic.keys()

dict_keys(['list_bbox.txt', 'list_category_img.txt', 'list_landmarks.txt'])

In [4]:
df = pd.merge(pd.merge(dic['list_bbox.txt'], dic['list_category_img.txt'], on='image_name'), dic['list_landmarks.txt'],
              on='image_name')

Replace category label with actual category text

In [5]:
cat = pd.read_csv('list_category_cloth.txt', sep=r'\s+', skiprows=1)
cat['list_index'] = range(1, 51)
cat['list_index'] = cat['list_index']
cat

,category_name,category_type,list_index
0,Anorak,1,1
1,Blazer,1,2
2,Blouse,1,3
3,Bomber,1,4
4,Button-Down,1,5
5,Cardigan,1,6
6,Flannel,1,7
7,Halter,1,8
8,Henley,1,9
9,Hoodie,1,10


In [6]:
df['category_label'] = df['category_label'].map(dict(zip(cat['list_index'], cat['category_name'])))

In [7]:
df.head()

,image_name,x_1,y_1,x_2,y_2,category_label,clothes_type,landmark_visibility_1,landmark_location_x_1,landmark_location_y_1,...,landmark_location_y_5,landmark_visibility_6,landmark_location_x_6,landmark_location_y_6,landmark_visibility_7,landmark_location_x_7,landmark_location_y_7,landmark_visibility_8,landmark_location_x_8,landmark_location_y_8
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,72,79,232,273,Blouse,1,0,146,102,...,229.0,0.0,177.0,232.0,NaN,NaN,NaN,NaN,NaN,NaN
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,67,59,155,161,Blouse,1,0,107,67,...,150.0,0.0,134.0,152.0,NaN,NaN,NaN,NaN,NaN,NaN
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,65,65,156,200,Blouse,1,0,101,79,...,188.0,0.0,118.0,185.0,NaN,NaN,NaN,NaN,NaN,NaN
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,51,62,167,182,Blouse,1,0,96,74,...,148.0,0.0,120.0,148.0,NaN,NaN,NaN,NaN,NaN,NaN
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,46,88,166,262,Blouse,1,1,102,106,...,245.0,0.0,137.0,246.0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
cloth = pd.read_csv('list_attr_cloth.txt', sep=r'\s{2,}', skiprows=1)

C:\Users\Kenji\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [9]:
attr = pd.read_csv('list_attr_img.txt', sep=r'\s+', skiprows=2, na_values=[-1, 0], header=None)
attr.columns = ['image_name'] + cloth['attribute_name'].to_list()

In [10]:
attr.shape

(289222, 1001)

In [11]:
attr.head()

,image_name,a-line,abstract,abstract chevron,abstract chevron print,abstract diamond,abstract floral,abstract floral print,abstract geo,abstract geo print,...,zeppelin,zig,zigzag,zip,zip-front,zip-pocket,zip-up,zipped,zipper,zippered
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
master = df.merge(attr, on='image_name', how='inner')

In [28]:
master.to_csv('all_data.csv', index=False)